In [1]:
import re
import geocoder
import numpy as np
import import_ipynb
import pandas as pd
import screen_fails as sf
import central_functions as cf
from geopy.geocoders import Nominatim
from data_files import indexed_files, visit
from central_functions import read_files, handle_nan

importing Jupyter notebook from screen_fails.ipynb
importing Jupyter notebook from data_files.ipynb
importing Jupyter notebook from central_functions.ipynb
C:/Users/dgao/Box/Intern Data Models/461-012/dta_demographics.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_iex_proc.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_sbj_participation.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_subject_termination.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_time_table.txt
C:/Users/dgao/Box/Intern Data Models/461-012/elig_enroll_log.pdf
C:/Users/dgao/Box/Intern Data Models/461-012/myk461-012_visit.csv
C:/Users/dgao/Box/Intern Data Models/461-012/udp_pi_eligibility_statement.txt
importing Jupyter notebook from patient_visits.ipynb
0 C:/Users/dgao/Box/Intern Data Models/224-001/224_001_elig_enroll_log.csv
1 C:/Users/dgao/Box/Intern Data Models/224-001/elig_enrollment_log.pdf
2 C:/Users/dgao/Box/Intern Data Models/461-005/elig.sas7bdat
3 C:/Users/dgao/Box/Intern Data Models/

C:\Users\dgao\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


SF before check:  (709, 12)
SF after check/dropping 'randomized' patients:  (677, 12)
Number of patient IDs to add to visits:  290
Num of rows in visits BEFORE rows added from SF:  9366
This is the new # of rows in visits (should be 305 more) 9656


In [2]:
indexed_files

[(0, 'C:/Users/dgao/Box/Intern Data Models/224-001/224-001.xlsx'),
 (1,
  'C:/Users/dgao/Box/Intern Data Models/224-001/224_001_elig_enroll_log.csv'),
 (2, 'C:/Users/dgao/Box/Intern Data Models/224-001/224_001_visits.xlsx'),
 (3, 'C:/Users/dgao/Box/Intern Data Models/224-001/dm.xlsx'),
 (4, 'C:/Users/dgao/Box/Intern Data Models/224-001/elig_enrollment_log.pdf'),
 (5, 'C:/Users/dgao/Box/Intern Data Models/461-004/dm1.sas7bdat'),
 (6, 'C:/Users/dgao/Box/Intern Data Models/461-004/dscomp2.sas7bdat'),
 (7, 'C:/Users/dgao/Box/Intern Data Models/461-004/subject.sas7bdat'),
 (8, 'C:/Users/dgao/Box/Intern Data Models/461-004/visit.sas7bdat'),
 (9, 'C:/Users/dgao/Box/Intern Data Models/461-005/dm1.sas7bdat'),
 (10, 'C:/Users/dgao/Box/Intern Data Models/461-005/dm2.sas7bdat'),
 (11, 'C:/Users/dgao/Box/Intern Data Models/461-005/dsrand1.sas7bdat'),
 (12, 'C:/Users/dgao/Box/Intern Data Models/461-005/elig.sas7bdat'),
 (13, 'C:/Users/dgao/Box/Intern Data Models/461-005/eligibility.sas7bdat'),
 (14,

In [3]:
# read_files(indexed_files[47][1]).columns
indexed_files[41]

(41, 'C:/Users/dgao/Box/Intern Data Models/461-012/dta_sbj_participation.txt')

In [4]:
# Data mostly taken from visit files
s_224_001 = read_files(indexed_files[2][1],skiprows=1)
s_461_004 = read_files(indexed_files[8][1])
s_461_005 = read_files(indexed_files[15][1])
s_461_006 = read_files(indexed_files[24][1])
s_461_007 = read_files(indexed_files[31][1])
s_461_008 = read_files(indexed_files[38][1])
s_461_012 = read_files(indexed_files[41][1])
s_461_013 = read_files(indexed_files[47][1])
s_461_014 = read_files(indexed_files[55][1])
s_461_015 = read_files(indexed_files[62][1])
s_461_018 = read_files(indexed_files[65][1])
s_491_003_mad = read_files(indexed_files[69][1])
s_491_003_sad = read_files(indexed_files[73][1])

In [5]:
def run_all_for_sites(df):
    df = subset_for_sites(df)
    df = rename_cols_for_sites(df)
    return df

In [6]:
def set_countries(df):
    for i,row in df.iterrows():
        if df.loc[i,'site_id'] < 200:
            df.loc[i,'country'] = 'USA'
        if df.loc[i,'site_id'] >= 200 and df.loc[i,'site_id'] < 296:
            df.loc[i,'country'] = 'ESP'
        if df.loc[i,'site_id'] >= 296 and df.loc[i,'site_id'] < 300:
            df.loc[i,'country'] = 'POL'
        if df.loc[i,'site_id'] > 300 and df.loc[i,'site_id'] < 307:
            df.loc[i,'country'] = 'ITA'
        if df.loc[i,'site_id'] > 396 and df.loc[i,'site_id'] < 400:
            df.loc[i,'country'] = 'PRT'
        if df.loc[i,'site_id'] > 400 and df.loc[i,'site_id'] < 408:
            df.loc[i,'country'] = 'ISR'
        if df.loc[i, 'site_id'] >= 500 and df.loc[i, 'site_id'] < 600:
            df.loc[i,'country'] = 'DEU'
        if df.loc[i,'site_id'] > 600 and df.loc[i,'site_id'] < 605:
            df.loc[i,'country'] = 'GBR'
        if df.loc[i,'site_id'] > 700 and df.loc[i,'site_id'] < 705:
            df.loc[i,'country'] = 'NLD'
        if df.loc[i,'site_id'] > 795 and df.loc[i,'site_id'] < 800:
            df.loc[i,'country'] = 'FRA'
        if df.loc[i,'site_id'] > 895 and df.loc[i,'site_id'] < 900:
            df.loc[i,'country'] = 'DNK'
        if df.loc[i,'site_id'] > 900 and df.loc[i,'site_id'] < 903:
            df.loc[i,'country'] = 'CZE'
        if df.loc[i,'site_id'] > 995 and df.loc[i,'site_id'] < 1000:
            df.loc[i,'country'] = 'BEL'
    return df

In [7]:
def concat_for_sites(list_of_dfs):
    for i in range(len(list_of_dfs)):
        list_of_dfs[i] = list_of_dfs[i].rename(columns={'SiteName':'site_name','Site Name':'site_name','Site':'site_name','SiteCode':'site_id','Site_ID':'site_id','SiteNumber':'site_id','SITE':'site_id','Site ID':'site_id','SITEID':'site_id','Country':'country','COUNTRY':'country'},errors='ignore')
        list_of_dfs[i]['site_id'] = list_of_dfs[i].site_id.apply(lambda x: int(x) if type(x) == float or type(x) == str else x)
        list_of_dfs[i] = list_of_dfs[i].drop_duplicates('site_id')
    df = pd.concat(list_of_dfs,axis=0)
    return df

In [8]:
# Data mostly taken from visit files
s_224_001 = read_files(indexed_files[2][1],skiprows=1)
s_461_004 = read_files(indexed_files[8][1])
s_461_005 = read_files(indexed_files[16][1])
s_461_006 = read_files(indexed_files[24][1])
s_461_007 = read_files(indexed_files[31][1])
s_461_008 = read_files(indexed_files[38][1])
# s_461_012 = read_files(indexed_files[41][1])
s_461_013 = read_files(indexed_files[47][1])
# s_461_014 = read_files(indexed_files[55][1])
s_461_015 = read_files(indexed_files[62][1])
# s_461_018 = read_files(indexed_files[65][1])
s_491_003_mad = read_files(indexed_files[69][1])
s_491_003_sad = read_files(indexed_files[73][1])

In [9]:
def extract_site_cols(s_461_0):
    cols = []
    for each in s_461_0.columns:
        if 'site' in each.lower():
            cols.append(each)
    return s_461_0[cols]
        
si_224_001 = extract_site_cols(s_224_001)
si_461_004 = extract_site_cols(s_461_004)
si_461_005 = extract_site_cols(s_461_005)
si_461_006 = extract_site_cols(s_461_006)
si_461_007 = extract_site_cols(s_461_007)
si_461_008 = extract_site_cols(s_461_008)
si_461_012 = extract_site_cols(s_461_012)
si_461_013 = extract_site_cols(s_461_013)
si_461_014 = extract_site_cols(s_461_014)
si_461_015 = extract_site_cols(s_461_015)
si_461_018 = extract_site_cols(s_461_018)
si_491_003_mad = extract_site_cols(s_491_003_mad)
si_491_003_sad = extract_site_cols(s_491_003_sad)

In [10]:
site_224_001 = s_224_001[['SiteName','SiteCode']]
site_224_001['study_id'] = 'myk_224_001'
site_461_004 = s_461_004[['Site','SiteNumber']]
site_461_004['study_id'] = 'myk_461_004'
site_461_004['SiteNumber'] = site_461_004.SiteNumber.apply(lambda x: x[-3:])
site_461_005 = s_461_005[['Site','SiteNumber']]
site_461_005['study_id'] = 'myk_461_005'
site_461_006 = s_461_006[['Site','SiteNumber']]
site_461_006['study_id'] = 'myk_461_006'
site_461_007 = s_461_007[['Site','SiteNumber']]
site_461_007['study_id'] = 'myk_461_007'
site_461_008 = s_461_008[['Site','SiteNumber']]
site_461_008['study_id'] = 'myk_461_008'
# site_461_012 = s_461_012[['study_no_sponsor']]
site_461_013 = s_461_013[['COUNTRY','SITEID']]
site_461_013['study_id'] = 'myk_461_013'
# site_461_014 = s_461_014[['study_no_sponsor']]
site_461_015 = s_461_015[['Site','SiteNumber']]
site_461_015['SiteNumber'] = site_461_015.SiteNumber.apply(lambda x: x[-3:])
site_461_015['study_id'] = 'myk_461_015'
# site_461_018 = site_461_018[['STUDY_NO_SPONSOR']] -- may not have been activated
site_491_003_mad = s_491_003_mad[['SITE']]
site_491_003_mad['study_id'] = 'myk_491_003_mad'
site_491_003_sad = s_491_003_sad[['SITE']]
site_491_003_sad['study_id'] = 'myk_491_003_sad'

<ipython-input-10-9beb32944cb1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  site_224_001['study_id'] = 'myk_224_001'
<ipython-input-10-9beb32944cb1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  site_461_004['study_id'] = 'myk_461_004'
<ipython-input-10-9beb32944cb1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [11]:
country_sites = concat_for_sites([site_224_001,site_461_004,site_461_005,site_461_006,site_461_007,site_461_008,site_461_013,site_461_015,site_491_003_mad,site_491_003_sad])

In [12]:
# set aside is for 461_015 because the site_ids are duplicate but do not match site name to ID
set_aside = country_sites[country_sites.study_id == 'myk_461_015']
COUNTRY_SITES = pd.concat([country_sites.drop_duplicates('site_id'),set_aside],axis=0)
COUNTRY_SITES['site_name'] = COUNTRY_SITES.site_name.apply(handle_nan)
COUNTRY_SITES['site_name'] = COUNTRY_SITES.site_name.apply(lambda x: re.sub('[0-9]+\s*\-*\s*',"",x))
COUNTRY_SITES = COUNTRY_SITES.reset_index(drop=True)
COUNTRY_SITES = set_countries(COUNTRY_SITES)
COUNTRY_SITES.loc[0,'country'] = 'AUS'
# COUNTRY_SITES = COUNTRY_SITES.append([['',133,'myk_461_005',''],['',502,'myk_461_005','']],ignore_index=True)
COUNTRY_SITES = COUNTRY_SITES.append({'site_name':'Ochsner Medical Center','site_id':133,'country':'USA','study_id':'myk_461_005'},ignore_index=True)
COUNTRY_SITES = COUNTRY_SITES.append({'site_name':'Universitatsklinikum Schleswig-Holstein','site_id':502,'country':'DEU','study_id':'myk_461_005'},ignore_index=True)

In [13]:
# COUNTRY_SITES = COUNTRY_SITES.drop('study_id',axis=1)
COUNTRY_SITES.loc[81,'site_id'] = 1152
COUNTRY_SITES.loc[80,'site_id'] = 1151
COUNTRY_SITES[COUNTRY_SITES.site_id=='']
COUNTRY_SITES.loc[78,['site_name','site_id']] = ['Covance Clinical Research Unit',1130]
COUNTRY_SITES.loc[79,'site_name'] = 'Ohio State University Medical Center'
COUNTRY_SITES.loc[48,'site_name'] = 'Hopital Europeen Georges Pompidou'
COUNTRY_SITES.loc[51,'site_name'] = 'Charite - Universitaetsmedizin Berlin'

C:\Users\dgao\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [14]:
COUNTRY_SITES = COUNTRY_SITES.append({'site_name':'Parexel Clinical Research Unit','site_id':1120,'study_id':'myk_461_012','country':'USA'},ignore_index=True)
COUNTRY_SITES = COUNTRY_SITES.reset_index()
COUNTRY_SITES = COUNTRY_SITES.rename(columns={'index':'SITES_idx'})

In [15]:
# To be joined with COUNTRY_SITES to weed out duplicates and assign non exisiting site IDs
sites_491_003 = read_files('C:/Users/dgao/Desktop/MYK491003_SiteList.xlsx' ) [['ID','Site - Name','PI - Country']]
sites_491_003 = sites_491_003.replace({'United States':'USA','Germany':'DEU','United Kingdom':'GBR','Netherlands':'NLD','Poland':'POL','France':'FRA','Sweden':'SWE'})
sites_491_003['study_id_to_add'] = 'myk_491_003'
sites_491_003 = sites_491_003.reset_index()
sites_491_003 = sites_491_003.rename(columns={'index':'myk_491_idx'})
sites_491_003['myk_491_idx'] = sites_491_003.myk_491_idx.apply(lambda x: int(x))

In [16]:
test = pd.merge(COUNTRY_SITES,sites_491_003,how='outer',left_on='country',right_on='PI - Country')
test = test.apply(lambda x: x.astype(str).str.lower())

In [17]:
# This function implements a rough algorithm to ID degrees-of-matching strings using a proportion of 
# similar words out of a total corpus of words
def return_similar(df,pct_match_lower,pct_match_upper=1):
    new_df = pd.DataFrame(columns=['site_name','site_id','study_id','country','ID','Site - Name','PI - Country'])
    for i,row in df.iterrows():
        row['site_name'] = row.site_name.split(' ')
        row['Site - Name'] = row['Site - Name'].split(' ')
        count = 0
        for word in row.site_name:
            if word in row['Site - Name']:
                count += 1
        if count*2/(len(row.site_name)+len(row['Site - Name'])) >= pct_match_lower and count*2/(len(row.site_name)+len(row['Site - Name'])) <= pct_match_upper:
            new_df = new_df.append(row)
    return new_df

def filter_sites(df1, df2,pct_match):
    df3 = pd.merge(df1,df2,how='outer',left_on='country',right_on='PI - country')
    while pct_match > 0.2:
        filter_sites(df1,return_similar(df3,pct_match),pct_match)     

In [18]:
sims = return_similar(test,0.4).reset_index()

# The index position in the sims DF where site name (roughly) matches between COUNTRY_SITES and 491_003
sim_idx = [0,2,3,30,37,38]
dups = sims.iloc[sim_idx,:]
dups['myk_491_idx'] = dups.myk_491_idx.apply(lambda x: int(float(x)))

<ipython-input-18-056f6cc95acb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dups['myk_491_idx'] = dups.myk_491_idx.apply(lambda x: int(float(x)))


In [19]:
idx_491 = [int(i) for i in range(sites_491_003.shape[0])]
idx_491_used = list(dups.myk_491_idx.values)
idx = set(idx_491)
idx_used = set(idx_491_used)
idx_left = idx-idx_used
sites_left = sites_491_003.iloc[list(idx_left),:].reset_index(drop=True)

# Dropped idx 0, because it is a matched site between 461-005 and 491-003
sites_left = sites_left.iloc[1:].drop(['myk_491_idx'],axis=1)
sites_left = sites_left.rename(columns={'ID':'orig_site_id','Site - Name':'site_name','PI - Country':'country','study_id_to_add':'study_id'})
sites_left['site_id'] = sites_left.orig_site_id.apply(lambda x: 491000+x)

SITES = pd.concat([COUNTRY_SITES,sites_left],axis=0)
SITES.loc[0,'site_id'] = 2241
SITES = SITES.reset_index(drop=True)

In [20]:
test2 = pd.merge(SITES,SITES,how='outer',on='country')
test2 = test2.apply(lambda x: x.astype(str).str.lower())
test2 = test2.rename(columns={'site_name_x':'site_name','site_name_y':'Site - Name'})
return_similar(test2,pct_match_lower=0.6,pct_match_upper=0.99)

,site_name,site_id,study_id,country,ID,Site - Name,PI - Country,SITES_idx_x,SITES_idx_y,orig_site_id_x,orig_site_id_y,site_id_x,site_id_y,study_id_x,study_id_y
153,"[duke, university, medical, center]",NaN,NaN,usa,NaN,"[columbia, university, medical, center]",NaN,4.0,9.0,nan,nan,106,117,myk_461_004,myk_461_005
158,"[duke, university, medical, center]",NaN,NaN,usa,NaN,"[university, of, texas, southwestern, medical,...",NaN,4.0,14.0,nan,nan,106,124,myk_461_004,myk_461_005
165,"[duke, university, medical, center]",NaN,NaN,usa,NaN,"[university, of, pittsburgh, medical, center]",NaN,4.0,21.0,nan,nan,106,127,myk_461_004,myk_461_005
171,"[duke, university, medical, center]",NaN,NaN,usa,NaN,"[university, of, washington, medical, center]",NaN,4.0,27.0,nan,nan,106,128,myk_461_004,myk_461_005
172,"[duke, university, medical, center]",NaN,NaN,usa,NaN,"[university, of, cincinnati, medical, center]",NaN,4.0,28.0,nan,nan,106,112,myk_461_004,myk_461_005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2361,"[ziv, medical, center]",NaN,NaN,isr,NaN,"[hadassah, medical, center]",NaN,38.0,37.0,nan,nan,402,406,myk_461_005,myk_461_005
2363,"[ziv, medical, center]",NaN,NaN,isr,NaN,"[barzilai, medical, center]",NaN,38.0,39.0,nan,nan,402,407,myk_461_005,myk_461_005
2366,"[barzilai, medical, center]",NaN,NaN,isr,NaN,"[kaplan, medical, center]",NaN,39.0,35.0,nan,nan,407,404,myk_461_005,myk_461_005
2368,"[barzilai, medical, center]",NaN,NaN,isr,NaN,"[hadassah, medical, center]",NaN,39.0,37.0,nan,nan,407,406,myk_461_005,myk_461_005


In [21]:
def create_site_labels(df):
    academic = ['university','groningen','newark beth israel','henry ford','heidelberg',"bartholomew's",'bispebjerg hospital','aarhus','universitets','erasme','columbia university','duke','brigham','nyu','stanford','ucsf','winthrop','universitario','tel aviv sourasky','universitaetsmedizin','uz antwerpen','erasmus mc','chru nantes','rangueil','experimental and clinical research center','collegium medicum uniwersytetu','groupe hospitalier piti']
    cro = ['d&a','parexel','nucleus network','covance','orlando clinical','jacksonville center for clinical','prism','medpace','center for clinical']
    non_profit = ['intermountain medical','yale','mayo clinic',"st. luke's university",'unity point','texas liver','ochsner']
    health_center = ['im m kopernika','cedars-sinai','carolinas healthcare','rabin medical','medical center','hospital da luz','st. louis heart and vascular cardiology']
    hospital = ['glasgow','kerckhoff-klinik','kaplan','houston methodist','onze-lieve-vrouwziekenhuis','georges pompidou','klinicke a experimentalni mediciny','hospital garcia','vseobecna fakultni nemocnice v praze']
#     ist = 
    for i,row in df.iterrows():
        for inst in academic:
            if inst in row.site_name.lower():
                df.loc[i,'site_type'] = 'academic'
        for org in cro:
            if org in row.site_name.lower():
                df.loc[i,'site_type'] = 'clinical research org'
        for ngo in non_profit:
            if ngo in row.site_name.lower():
                df.loc[i,'site_type'] = 'non profit'
        for center in health_center:
            if center in row.site_name:
                df.loc[i,'site_type'] = 'health center'
        for hosp in hospital:
            if hosp in row.site_name.lower():
                df.loc[i,'site_type'] = 'hospital'
        if 'intermountain medical' in row.site_name.lower():
            df.loc[i,'locale'] = 'Salt Lake City, UT'
        if 'university of michigan' in row.site_name.lower():
            df.loc[i,'locale'] = 'Ann Arbor, MI'
        if 'yale new haven' in row.site_name.lower():
            df.loc[i,'locale'] = 'New Haven, CT'
        if 'mayo clinic arizona' in row.site_name.lower():
            df.loc[i,'locale'] = 'Scottsdale, AZ'
        if 'oregon health and science' in row.site_name.lower():
            df.loc[i,'locale'] = 'Portland, OR'
        if 'university of pennsylvania' in row.site_name.lower():
            df.loc[i,'locale'] = 'Philadelphia, PA'
        if 'duke' in row.site_name.lower():
            df.loc[i,'locale'] = 'Durham, NC'
        if 'washington university' in row.site_name.lower():
            df.loc[i,'locale'] = 'St. Louis, MO'
        if 'brigham' in row.site_name.lower():
            df.loc[i,'locale'] = 'Boston, MA'
        if 'cedars-sinai heart' in row.site_name.lower():
            df.loc[i,'locale'] = 'Los Angeles, California'
        if 'columbia' in row.site_name.lower():
            df.loc[i,'locale'] = 'Manhattan, NY'
        if 'houston methodist' in row.site_name.lower():
            df.loc[i,'locale'] = 'Houston, TX'
        if 'northwestern' in row.site_name.lower():
            df.loc[i,'locale'] = 'Chicago, IL'
        if "st. luke's university" in row.site_name.lower():
            df.loc[i,'locale'] = 'Orwigsburg, PA'
        if 'university of iowa' in row.site_name.lower():
            df.loc[i,'locale'] = 'Iowa City, IA'
        if 'texas southwestern' in row.site_name.lower():
            df.loc[i,'locale'] = 'Dallas, TX'
        if 'university of utah' in row.site_name.lower():
            df.loc[i,'locale'] = 'Salt Lake City, UT'
        if 'university of virginia' in row.site_name.lower():
            df.loc[i,'locale'] = 'Charlottesville, VA'
        if 'virginia commonwealth' in row.site_name.lower():
            df.loc[i,'locale'] = 'Richmond, VA'
        if 'winthrop cardiology' in row.site_name.lower():
            df.loc[i,'locale'] = 'Mineola, NY'
        if 'carolinas healthcare' in row.site_name.lower():
            df.loc[i,'locale'] = 'Morganton, NC'
        if 'texas houston' in row.site_name.lower():
            df.loc[i,'locale'] = 'Houston, TX'
        if 'university of pittsburgh' in row.site_name.lower():
            df.loc[i,'locale'] = 'Pittsburgh, PA'
        if 'mayo clinic jacksonville' in row.site_name.lower():
            df.loc[i,'locale'] = 'Jacksonville, FL'
        if 'nyu langone' in row.site_name.lower():
            df.loc[i,'locale'] = 'Manhattan, NY'
        if 'stanford' in row.site_name.lower():
            df.loc[i,'locale'] = 'Stanford, CA'
        if 'ucsf' in row.site_name.lower():
            df.loc[i,'locale'] = 'San Francisco, California'
        if 'university of washington medical' in row.site_name.lower():
            df.loc[i,'locale'] = 'Seattle, WA'
        if 'university of cincinnati' in row.site_name.lower():
            df.loc[i,'locale'] = 'Cincinnati, OH'
        if 'hospital universitario ramon' in row.site_name.lower():
            df.loc[i,'locale'] = 'Madrid'
        if 'hospital universitario puerta de hierro' in row.site_name.lower():
            df.loc[i,'locale'] = 'Madrid'
        if 'nucleus network' in row.site_name.lower():
            df.loc[i,'locale'] = 'Melbourne'
        if 'hospital universitario virgen macarena' in row.site_name.lower():
            df.loc[i,'locale'] = 'Sevilla'
        if 'hospital universitario virgen de la arrixaca' in row.site_name.lower():
            df.loc[i,'locale'] = 'Murcia'
        if 'tel aviv sourasky' in row.site_name.lower():
            df.loc[i,'locale'] = 'Tel Aviv'
        if 'kaplan' in row.site_name.lower():
            df.loc[i,'locale'] = 'Rehovot'
        if 'rabin medical' in row.site_name.lower():
            df.loc[i,'locale'] = 'Petah Tikva'
        if 'hadassah' in row.site_name.lower():
            df.loc[i,'locale'] = 'Jerusalem' 
        if 'ziv' in row.site_name.lower():
            df.loc[i,'locale'] = 'Safed'
        if 'barzilai' in row.site_name.lower():
            df.loc[i,'locale'] = 'Ashkelon'
        if 'chaim sheba' in row.site_name.lower():
            df.loc[i,'locale'] = 'Ramat Gan'
        if 'pital erasme' in row.site_name.lower():
            df.loc[i,'locale'] = 'Brussels'
        if 'onze-lieve-vrouwziekenhuis' in row.site_name.lower():
            df.loc[i,'locale'] = 'Aalst'
        if 'uz antwerpen' in row.site_name.lower():
            df.loc[i,'locale'] = 'Edegem'
        if 'erasmus mc' in row.site_name.lower():
            df.loc[i,'locale'] = 'Rotterdam'
        if 'maastricht university' in row.site_name.lower():
            df.loc[i,'locale'] = 'Maastricht'
        if 'chru nantes' in row.site_name.lower():
            df.loc[i,'locale'] = 'Nantes'
        if 'pital de rangueil' in row.site_name.lower():
            df.loc[i,'locale'] = 'Toulouse'
        if 'hopital europeen georges pompidou' in row.site_name.lower():
            df.loc[i,'locale'] = 'Paris'  
        if 'cardiologicum dresden und pirna' in row.site_name.lower():
            df.loc[i,'locale'] = ''
        if 'charite - universitaetsmedizin berlin' in row.site_name.lower():
            df.loc[i,'locale'] = 'Berlin'
        if 'campus buch - experimental and clinic' in row.site_name.lower():
            df.loc[i,'locale'] = 'Berlin'
        if 'GÃ¶ttingen Germany' in row.site_name:
            df.loc[i,'locale'] = 'Gottingen'
        if 'institut klinicke a experimentalni mediciny' in row.site_name.lower():
            df.loc[i,'locale'] = 'Prague'
        if 'vseobecna fakultni nemocnice v praze' in row.site_name.lower():
            df.loc[i,'locale'] = 'Prague'
        if 'Bispebjerg Hospital' in row.site_name:
            df.loc[i,'locale'] = 'Copenhagen'
        if 'hospital garcia de orta' in row.site_name.lower():
            df.loc[i,'locale'] = 'Almada'
        if 'hospital da luz' in row.site_name.lower():
            df.loc[i,'locale'] = 'Lisboa'
        if 'azienda ospedaliera universitaria careggi' in row.site_name.lower():
            df.loc[i,'locale'] = 'Florence'
        if 'kardio klinika bryn' in row.site_name.lower():
            df.loc[i,'locale'] = 'Katowice'
        if 'Instytut Kardiologii im Prymasa Tysiaclecia Ka' in row.site_name:
            df.loc[i,'locale'] = 'Warszawa'
        if 'Collegium Medicum Uniwersytetu Jagiellonskiego' in row.site_name:
            df.loc[i,'locale'] = 'Krakow'
        if 'UniversitÃ¤tsklinikum Heidelberg' in row.site_name:
            df.loc[i,'locale'] = 'Heidelberg'
        if 'Parexel Clinical Research Unit' in row.site_name:
            df.loc[i,'locale'] = 'Glendale, California'
        if 'Hospital Universitario A CoruÃ±a' in row.site_name:
            df.loc[i,'locale'] = 'Coruna'
        if 'Groupe Hospitalier PitiÃ© SalpÃ©triÃ¨' in row.site_name:
            df.loc[i,'locale'] = 'Paris'
        if 'Aarhus Universite' in row.site_name:
            df.loc[i,'locale'] = 'Aarhus'
        if 'Szpital Kliniczny' in row.site_name:
            df.loc[i,'locale'] = 'Poznan'
        if 'Covance Clinical' in row.site_name:
            df.loc[i,'locale'] = 'Dallas, TX'
        if 'Odense Universitetshospital' in row.site_name:
            df.loc[i,'locale'] = 'Odense'
        if 'University Hospital of Wales' in row.site_name:
            df.loc[i,'locale'] = 'Cardiff, Wales'
        if "St Bartholomew's Hospital" in row.site_name:
            df.loc[i,'locale'] = 'London'
        if 'Methodist University Hospital' in row.site_name:
            df.loc[i,'locale'] = 'Memphis, TN'
        if 'Kerckhoff-Klinik-Forschungs-GmbH' in row.site_name:
            df.loc[i,'locale'] = 'Bad Nauheim'
        if 'Henry Ford Hospital' in row.site_name:
            df.loc[i,'locale'] = 'Detroit, MI'
        if 'The Pennsylvania State University (Penn State)' in row.site_name:
            df.loc[i,'locale'] = 'Hershey, PA'
        if 'University of Maryland Medical Center' in row.site_name:
            df.loc[i,'locale'] = 'Baltimore, MD'
        if 'Unity Point Health - Heart and Vascular Institute' in row.site_name:
            df.loc[i,'locale'] = 'Madison, WI'
        if 'Ohio State University Medical Center' in row.site_name:
            df.loc[i,'locale'] = 'Columbus, OH'
        if 'Orlando Clinical Research Center' in row.site_name:
            df.loc[i,'locale'] = 'Orlando, FL'
        if 'The Texas Liver Institute' in row.site_name:
            df.loc[i,'locale'] = 'San Antonio, TX'
        if 'Ochsner Medical Center' in row.site_name:
            df.loc[i,'locale'] = 'Baton Rouge, LA'
        if 'Universitatsklinikum Schleswig-Holstein' in row.site_name:
            df.loc[i,'locale'] = 'Kiel'
        if 'Jacksonville Center for Clinical Research' in row.site_name:
            df.loc[i,'locale'] = 'Jacksonville, FL'
        if 'Prism Research' in row.site_name:
            df.loc[i,'locale'] = 'Saint Paul, MN'
        if 'Medpace Clinical Pharmacology, LLC' in row.site_name:
            df.loc[i,'locale'] = 'Cincinnati, OH'
        if 'Tennessee Center for Clinical Trials' in row.site_name:
            df.loc[i,'locale'] = 'Nashville, TN'
        if 'Newark Beth Israel Medical Center - Cardiology' in row.site_name:
            df.loc[i,'locale'] = 'Newark, NJ'
        if 'St. Louis Heart and Vascular Cardiology' in row.site_name:
            df.loc[i,'locale'] = 'St. Louis, MO'
        if 'Queen Elizabeth University Hospital - Glasgow' in row.site_name:
            df.loc[i,'locale'] = 'Glasgow, Scotland'
        if 'Karolinska University Hospital' in row.site_name:
            df.loc[i,'locale'] = 'Solna, Sweden'
        if 'D&A Research' in row.site_name:
            df.loc[i,'locale'] = 'Houston, TX'
        if 'Groningen UMC' in row.site_name:
            df.loc[i,'locale'] = 'Groningen'
        if 'Wojewodzki Szpital Specjalistyczny we Wroclawiu, Oddzial Kardiologiczny z Pododdzialem Intensywnego Nadzoru Kardiologicznego i Pododdzialem Leczenia Zaburzen Rytmu Serca' in row.site_name:
            df.loc[i,'locale'] = ''
        if 'Wojewodzki Szpital Specjalistyczny Im M Kopernika' in row.site_name:
            df.loc[i,'locale'] = 'Lodz, POL'
            
#         DEAN VERIFY WHAT THIS SITE ACTUALLY IS -- Indianapolis, IN(?)
        if 'Saint Vincent Hospital and Health Services' in row.site_name:
            df.loc[i,'locale'] = 'Indianapolis, IN'
#         DEAN VERIFY WHAT THIS SITE ACTUALLY IS
        if 'â\x80\x93 Spectrum Health' in row.site_name:
            df.loc[i,'locale'] = ''       
    for i,row in df.iterrows():
        locale2 = row.locale.split(',')
        if len(locale2)>1:
            df.loc[i,'city'] = locale2[0]
            df.loc[i,'state'] = locale2[1]
        else:
            df.loc[i,'city'] = locale2[0]
    return df
        
SITES = create_site_labels(SITES)
SITES = SITES.replace({'Vseobecna fakultni nemocnice v Praze':'General University Hospital in Prague','Kardio Klinika BrynÃ³w':'Cardio Clinic Brynow','Hospital Universitario A CoruÃ±a':'Hospital Universitario A Coruna','Groupe Hospitalier PitiÃ© SalpÃ©triÃ¨re':'University Hospitals Pitié Salpêtrière - Charles Foix','HÃ´pital de Rangueil':'Hopital de Rangueil','â\x80\x93 Spectrum Health':'Spectrum Health','UniversitÃ¤tsklinikum Heidelberg':'University Hospital Heidelberg'})


In [22]:
SITES[SITES.locale=='']
# SITES[SITES.site_name.str.contains('Ochsner')]

,SITES_idx,site_name,site_id,study_id,country,orig_site_id,site_type,locale,city,state
50,50.0,Cardiologicum Dresden und Pirna,504,myk_461_005,DEU,NaN,NaN,,,NaN
68,68.0,Spectrum Health,141,myk_461_005,USA,NaN,NaN,,,NaN
95,NaN,Wojewodzki Szpital Specjalistyczny we Wroclawi...,491601,myk_491_003,POL,601.0,NaN,,,NaN


In [23]:
def gen_lat_lng(df):
    lat_lng = df[['locale']]
    locator = Nominatim(user_agent='myGeocoder')
    for i,row in df.iterrows():
        if row.locale == '':
            lat_lng.loc[i,'lat'] = 0
            lat_lng.loc[i,'lng'] = 0
        else:
            location = locator.geocode(row.locale)
            lat_lng.loc[i,'latitude'] = location[1][0]
            lat_lng.loc[i,'longitude'] = location[1][1]
    return lat_lng

lat_lng = gen_lat_lng(SITES)
lat_lng

C:\Users\dgao\Anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\dgao\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,locale,latitude,longitude,lat,lng
0,Melbourne,-37.814218,144.963161,NaN,NaN
1,"Scottsdale, AZ",33.494219,-111.926018,NaN,NaN
2,"Portland, OR",45.520247,-122.674195,NaN,NaN
3,"New Haven, CT",41.308214,-72.925052,NaN,NaN
4,"Durham, NC",35.996653,-78.901805,NaN,NaN
...,...,...,...,...,...
92,"Solna, Sweden",59.361367,18.001369,NaN,NaN
93,"Houston, TX",29.758938,-95.367697,NaN,NaN
94,Groningen,53.219065,6.568008,NaN,NaN
95,,NaN,NaN,0.0,0.0


In [24]:
SITES.to_csv('C:/Users/dgao/Desktop/Spotfire_DATA/all_sites.csv')
lat_lng.to_csv('C:/Users/dgao/Desktop/Spotfire_DATA/lat_lng_table.csv')

In [25]:
SITES[SITES.site_name.str.contains('Prism')]

,SITES_idx,site_name,site_id,study_id,country,orig_site_id,site_type,locale,city,state
86,NaN,Prism Research,491106,myk_491_003,USA,106.0,clinical research org,"Saint Paul, MN",Saint Paul,MN


In [26]:
def find_loc_mult_sites(groupby_df):
    mult_loc_sites = []
    for i,row in groupby_df.iterrows():
        if groupby_df.shape[0] > 1:
            mult_loc_sites.append(i)
    return mult_loc_sites
    
list_mult_sites = SITES.groupby('city').apply(find_loc_mult_sites)
idx_mult_sites = [idx for sublist in list_mult_sites.values for idx in sublist]
loc_with_mult_sites = SITES.iloc[idx_mult_sites,:]

idx_single_sites = list(set(SITES.index.values) - set(idx_mult_sites))
idx_single_sites
loc_with_single_site = SITES.iloc[idx_single_sites,:]

In [27]:
loc_with_mult_sites.to_csv('C:/Users/dgao/Desktop/Spotfire_DATA/test_loc_mult_sites.csv')
loc_with_single_site.to_csv('C:/Users/dgao/Desktop/Spotfire_DATA/test_loc_single_site.csv')

In [30]:
loc_with_single_site.head(60)
loc_with_mult_sites
SITES[SITES.site_id==101]

,SITES_idx,site_name,site_id,study_id,country,orig_site_id,site_type,locale,city,state
3,3.0,Yale New Haven Hospital,101,myk_461_004,USA,NaN,non profit,"New Haven, CT",New Haven,CT


In [29]:
246+32
32/278

0.11510791366906475